In [ ]:
from transformer.transformer import Transformer
from lstm.lstm import LSTM
from gru.gru import GRU
from LinearNetwork.linear import Linear
import torch.optim as optim
import torch.nn as nn
from torch.autograd import Variable
from tqdm import tqdm
import sklearn.metrics as metrics
from sklearn.ensemble import RandomForestClassifier
from configparser import Interpolation
import torch
import numpy as np
from torch.utils.data import Dataset
import pandas as pd
from torch.utils.data import DataLoader

seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True

In [ ]:
class Dataset_Classification(Dataset):
    def __init__(self, csv):
        self.csv = csv

    def __len__(self):
        return len(self.csv['image'])
    
    def __getitem__(self, idx):
        input = torch.Tensor(self.csv['image'][idx])
        label = torch.Tensor([self.csv['label'][idx]])
        return input, label

In [ ]:
test = np.load("gru/test.npy", allow_pickle=True).item()
test_dataset = Dataset_Classification(csv=test)
testloader = DataLoader(test_dataset, batch_size=1,
                            num_workers=0, pin_memory=True, shuffle=False)  

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
type = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor

model1 = Linear(in_channels=2048, embedding_channels=1024, hidden_channels=None, classes=1, depth=4, dim_reduction=True)
model2 = GRU(in_channels=2048, embedding_channels=1024, hidden_channels=512, classes=1, num_layers=1, dim_reduction=True)
model3 = LSTM(in_channels=2048, embedding_channels=1024, hidden_channels=512, classes=1, num_layers=1, dim_reduction=True)
model4 = Transformer(feature_dimensions=2048, dim_reduction=2, num_head=16, depth=1, act_layer=nn.GELU)

In [ ]:
checkpoint = torch.load("./LinearNetwork/best_ar_linear.pth.tar", map_location={'cuda:0':'cpu'})
model1.load_state_dict(checkpoint['state_dict'])
checkpoint = torch.load("./gru/best_ar_gru.pth.tar", map_location={'cuda:0':'cpu'})
model2.load_state_dict(checkpoint['state_dict'])
checkpoint = torch.load("./lstm/best_ar_lstm.pth.tar", map_location={'cuda:0':'cpu'})
model3.load_state_dict(checkpoint['state_dict'])
checkpoint = torch.load("./transformer/best_ar_trans.pth.tar", map_location={'cuda:0':'cpu'})
model4.load_state_dict(checkpoint['state_dict'])

In [ ]:
model1 = model1.to(device)
model2 = model2.to(device)
model3 = model3.to(device)
model4 = model4.to(device)

In [ ]:
y_test = []
y_pred = []
with torch.no_grad():
        for idx, data in enumerate(testloader):
            input, label = data
            input = Variable(input.type(type))
            label = Variable(label.type(type))
            y_test.extend(label.detach().cpu().numpy())
            output1 = model1(input)
            output2 = model2(input, device)
            output3 = model3(input, device)
            output4 = model4(input)
            output = np.mean([output1.detach().cpu().numpy(), output2.detach().cpu().numpy(), output3.detach().cpu().numpy(), output4.detach().cpu().numpy()])
            y_pred.extend([output])
        y_test = np.array(y_test)
        y_pred = np.array(y_pred)
        precision, recall, _ = metrics.precision_recall_curve(y_test, y_pred, pos_label=1)
        ap = metrics.auc(recall, precision)
        fpr_roc, tpr_roc, _ = metrics.roc_curve(y_test, y_pred, pos_label=1)
        ar = metrics.auc(fpr_roc, tpr_roc)

In [ ]:
print(f'ap : {ap}')
print(f'ar : {ar}')